# 16S Analysis of Caatinga marmoset gut microbiome

We analyzed 16S sequences from 69 marmoset fecal samples to examine gut microbial community composition. Sequence analysis was performed in QIIME (v2.2021.2) and downstream statistics were done in R (see 16S_Seq_Data_Analysis.R file).

## Importing sequences into QIIME

Demultiplexed sequences were imported using the manifest method. The manifest_builder.R script was used to create the Manifest.csv file below. We then examined the sequences to make sure the import proceeded as expected.

In [ ]:
qiime tools import --type 'SampleData[PairedEndSequencesWithQuality]' --input-path Manifest.csv
    --output-path paired-end-demux.qza --input-format PairedEndFastqManifestPhred33

qiime demux summarize --i-data paired-end-demux.qza --o-visualization paired-end-demux.qzv

## Trimming, quality-filtering, and ASV determination

DADA2 was used to denoise the 16S sequences.

In [ ]:
qiime dada2 denoise-paired --i-demultiplexed-seqs paired-end-demux.qza --p-trunc-len-f 290 --p-trunc-len-r 290 \
    --p-trim-left-f 20 --p-trim-left-r 20 --p-max-ee 5 --p-n-threads 8 --o-table table.qza \
    --o-representative-sequences rep-seqs.qza --o-denoising-stats dada2-stats.qza

qiime metadata tabulate --m-input-file data2-stats.qza --o-visualization dada2-stats.qzv

qiime feature-table summarize --i-table table.qza --o-visualization table.qzv \
    --m-sample-metadata-file caatinga_metadata.txt

## Generate phylogenetic tree

A phylogenetic tree was constructed using a QIIME pipeline incorporating fasttree and mafft.

In [ ]:
qiime phylogeny align-to-tree-mafft-fastree --i-sequences rep-seqs.qza --o-alignment aligned-rep-seqs.qza \
    --o-masked-alignment masked-aligned-rep-seqs.qza --o-tree unrooted-tree.qza --o-rooted-tree rooted-tree.qza

## Assign taxonomy

Taxonomy was assigned using a naive Bayesian classifier trained on the Greengenes 13_8 database. Sequences from mitochondria and chloroplast were removed.

In [ ]:
qiime feature-classifier classify-sklearn --i-classifier gg-13-8-99-nb-classifier-qiime2021-2.qza \
    --i-reads rep-seqs.qza --o-classification taxonomy.qza

qiime taxa filter-table --i-table table.qza --i-taxonomy taxonomy.qza --p-exclude mitochondria,chloroplast \
    --o-filtered-table table-nomito-nochloro.qza
    
qiime feature-table summarize --i-table table-nomito-nochloro.qza --o-visualization table-nomito-nochloro.qzv \
    --m-sample-metadata-file caatinga_metadata.txt

## Taxa barplots and feature tables

Barplot visualizations were created from the filtered feature table to visualize taxonomic assignments. Feature tables were collapsed at the level of phyla, family, and genus, and all tables were exported.

In [ ]:
qiime taxa barplot --i-table table-nomito-nochloro.qza --i-taxonomy taxonomy.qza \
    --m-metadata-file caatinga_metadata.txt --o-visualization taxa-plots-nomito-nochloro.qzv

qiime tools export --input-path table-nomito-nochloro.qza --output-path asv_table

qiime tools export --input-path taxonomy.qza --output-path asv_table

cp ./asv_table/taxonomy.tsv ./asv_table/biom-taxonomy.tsv

#edit header of biom-taxonomy .tsv to `#OTUID taxonomy confidence`

biom add-metadata -i ./asv_table/feature-table.biom  -o ./asv_table/feature-table-withtax.biom \
    --observation-metadata-fp ./asv_table/biom-taxonomy.tsv  --sc-separated taxonomy
    
biom convert -i ./asv_table/feature-table-withtax.biom -o ./asv_table/feature-table-full.tsv --to-tsv \
    --header-key taxonomy
    
qiime taxa collapse --i-table table-nomito-nochloro.qza --i-taxonomy taxonomy.qza --p-level 2 \
    --o-collapsed-table phyla_table.qza

qiime taxa collapse --i-table table-nomito-nochloro.qza --i-taxonomy taxonomy.qza --p-level 5 \
    --o-collapsed-table family_table.qza

qiime taxa collapse --i-table table-nomito-nochloro.qza --i-taxonomy taxonomy.qza --p-level 6 \
    --o-collapsed-table genus_table.qza

qiime tools export --input-path phyla_table.qza --output-path phyla_table
    
biom convert -i ./phyla_table/feature-table.biom -o ./phyla_table/phyla-table-full.tsv --to-tsv
    
qiime tools export --input-path family_table.qza --output-path family_table
    
biom convert -i ./family_table/feature-table.biom -o ./family_table/family-table-full.tsv --to-tsv 

qiime tools export --input-path genus_table.qza --output-path genus_table

biom convert -i ./genus_table/feature-table.biom -o ./genus_table/genus-table-full.tsv --to-tsv 

## Diversity analyses

QIIME's core diversity pipeline was used to calculate alpha and beta diversity. The feature table was rarefied to 10,000 ASVs per sample prior to calculation of diversity indices. Alpha diversity values and beta diversity distance matrices were exported for downstream analysis in R.

In [ ]:
qiime diversity core-metrics-phylogenetic --i-phylogeny rooted-tree.qza --i-table table-nomito-nochloro.qza 
    --p-sampling-depth 10000 --m-metadata-file caatinga_metadata.txt --output-dir core-metrics-results-10000

qiime tools export --input-path core-metrics-results-10000/faith_pd_vector.qza --output-path alpha_out

qiime tools export --input-path core-metrics-results-10000/shannon_vector.qza --output-path alpha_out

qiime tools export --input-path core-metrics-results-10000/observed_otus_vector.qza --output-path alpha_out

qiime tools extract --input-path core-metrics-results-10000/unweighted_unifrac_distance_matrix.qza 
    --output-path core-metrics-results-10000

qiime tools extract --input-path core-metrics-results-10000/weighted_unifrac_distance_matrix.qza 
    --output-path core-metrics-results-1000

qiime tools extract --input-path core-metrics-results-10000/bray_curtis_distance_matrix.qza 
    --output-path core-metrics-results-10000

qiime tools extract --input-path core-metrics-results-10000/jaccard_distance_matrix.qza 
    --output-path core-metrics-results-10000